# Use case 1: 



## Dataset: Breast cancer
## Goal: Automatically containerize a reproducible Machine Learning pipeline with the aim of being run on any platform.

### Input:

Three ML steps, dataset, and a list of requirements libraries.

- gathering.py, preprocessing.py, modeling.py

- breast-cancer.csv

- requirements.txt

    #### [gathering] -> [preprocessing] -> [modeling]

### Output:

- 1 docker image. It contains the platform where the workflow runs.
- 1 docker container. Based on the previous image, start up a dashboard with the workflow metrics.

    #### [image] - [container]

# Autodeploy


## Setup

**Input**: 

- Folder with the python files, data set and requirements.

**Output**: 

- Platform.
- Dashboard at http://localhost:8001 for metrics. 

In [3]:
from autodeploy.setup import setup


app_dir = '/home/guess/Desktop/autodeploy/examples/leaf'
workflow = {'gathering': 'gathering.py',
            'preprocessing': 'preprocessing.py',
            'modeling': 'modeling.py',
            'main': 'main.py'}

platform = setup.Setup(app_dir, workflow)

platform.build()
platform.run()

platform

27-Oct-19 21:05:20 -  INFO - Building platform, type: single.
27-Oct-19 21:05:20 -  INFO - Dockerfile was found.
27-Oct-19 21:05:20 -  WARNING - Image: app_single already exists.
27-Oct-19 21:05:20 -  INFO - Running platform, type: single.
27-Oct-19 21:05:20 -  ERROR - 409 Client Error: Conflict ("Conflict. The container name "/app_single" is already in use by container "8c597ef99360f1251126bfe6aa5b19ed08e9cf8891d71830c3e5a0ab28a565b8". You have to remove (or rename) that container to be able to reuse that name.")
27-Oct-19 21:05:20 -  ERROR - Container creation failed.


# Deploy

Run the steps.

- Input: Platform.
- Output: Results of running the workflow shown at http://localhost:8001/#/. 

In [8]:
from autodeploy.deploy import deploy

deployer = deploy.Deploy(platform)

# deployer.pipeline()
deployer.run_workflow(plat_container_name='app_single')
# deployer.deploy()


25-Oct-19 16:43:34 - Running workflow: type=single .
25-Oct-19 16:43:34 - Using given container: app_single
25-Oct-19 16:43:40 -  Main file (main.py) output:  Run matched, but is not FINISHED, so skipping (run_id=e48d762176914bfa807563d6dba8bc5c, status=FINISHED)
Run matched, but is not FINISHED, so skipping (run_id=8fd33bc89be1452e8ea568fbdf4f9fc7, status=FINISHED)
No matching run has been found.
2019/10/25 14:43:36 INFO mlflow.projects: === Created directory /tmp/tmp0w29dabq for downloading remote URIs passed to arguments of type 'path' ===
2019/10/25 14:43:36 INFO mlflow.projects: === Running command 'python gathering.py' in run with ID '1bd37a93280d4fd0859b2f1ce6542491' === 
   species  specimen_number  eccentricity  ...  third_moment  uniformity  entropy
0        1                1       0.72694  ...      0.005232    0.000275  1.17560
1        1                2       0.74173  ...      0.002708    0.000075  0.69659
2        1                3       0.76722  ...      0.000921    0.